# Disclaimer

It should be noted that the code provided in this Jupyter notebook is not perfect, as its primary purpose is to demonstrate the feasibility of the proposed framework presented in the paper titled

"Framework to Create Inventory Dataset for Disaster Behavior Analysis Using Google Earth Engine: A Case Study in Peninsular Malaysia for Historical Forest Fire Behavior Analysis"

Forests 2024, 15(6), 923; https://doi.org/10.3390/f15060923


# Prerequisite - Installing Geemap locally

In [1]:
# GEE Installation through Conda 
# conda create -n gee geopandas ipykernel python=3.9
# conda install -n base nb_conda_kernels
# conda install -n base conda-libmamba-solver
# conda config --set solver libmamba
# conda install -c conda-forge geemap localtileserver
# jupyter notebook 

In [2]:
# Upgrade geemap to the latest version
# geemap.update_package()

# Creating Single Polygon from Feature Collection to reduce computation
- Country Boundary (all_states) - level 1 can be downloaded from: https://data.humdata.org/dataset/cod-ab-mys
- Modify this section to apply the framework to other location



In [3]:
import ee
import geemap
import os
import pandas as pd
from geemap.datasets import DATA, get_metadata

In [4]:
Map = geemap.Map()

In [5]:
    #Adding Malaysia States
all_states = ee.FeatureCollection("projects/chewyeejian/assets/mys_adm2")


#  _____Location Selection in Malaysia  via Feature Collection_____
#  [ADM1_PCODE] = MY01-Johor | MY02-Kedah | MY03-Kelantan | MY04-W.P. Kuala Lumpur | MY06-Melaka | MY07-Negeri | MY08-Pahang | MY09-Perak | MY10-Perlis | MY11-Pulau Pinang | MY15-Terengganu | MY16-W.P. Putrajaya | MY17-Selangor
#  [ADM1_PCODE] = MY05-W.P. Labuan | MY12-Sabah | MY13-Sarawak
peninsular = all_states.filter(ee.Filter.Or(
                                    ee.Filter.eq('ADM1_PCODE',"MY01"),ee.Filter.eq('ADM1_PCODE',"MY02"),ee.Filter.eq('ADM1_PCODE',"MY03"),
                                    ee.Filter.eq('ADM1_PCODE',"MY04"),ee.Filter.eq('ADM1_PCODE',"MY06"),ee.Filter.eq('ADM1_PCODE',"MY07"),
                                    ee.Filter.eq('ADM1_PCODE',"MY08"),ee.Filter.eq('ADM1_PCODE',"MY09"),ee.Filter.eq('ADM1_PCODE',"MY10"),                                    
                                    ee.Filter.eq('ADM1_PCODE',"MY11"),ee.Filter.eq('ADM1_PCODE',"MY14"),ee.Filter.eq('ADM1_PCODE',"MY15"),
                                    ee.Filter.eq('ADM1_PCODE',"MY16"),ee.Filter.eq('ADM1_PCODE',"MY17")                                    
                                    ))

sabahsarawak_states = all_states.filter(ee.Filter.Or(
                                    ee.Filter.eq('ADM1_PCODE',"MY05"),ee.Filter.eq('ADM1_PCODE',"MY13"),ee.Filter.eq('ADM1_PCODE',"MY12")))

pahang_states = all_states.filter(ee.Filter.Or
                                  (ee.Filter.eq('ADM1_PCODE',"MY08")))
                                    
# Map.addLayer(all_states,{},"All States")
# Map.addLayer(peninsular,{},"Peninsular")
# Map.addLayer(sabahsarawak_states,{},"Sabah-Sarawak")

# Use the international boundary
# Malaysia = ee.FeatureCollection("USDOS/LSIB/2017").filterMetadata("COUNTRY_NA","equals","Malaysia")
# Malaysia = ee.FeatureCollection("USDOS/LSIB/2017").filterMetadata("COUNTRY_NA","equals","Australia")
# Map.addLayer(Malaysia,{},"malaysia")



## Parameters to Set - Location & Dates
- year set to 2021 year (coz we got rompin case study)
- location set to pahang first (before we scale to the entire peninsular) 

In [6]:
#  _____Location Selection in Malaysia  via Feature Collection_____
feature_selected_states = peninsular #multipolygon
# selected_states = peninsular #not used
# print(feature_selected_states,'feature_selected_states')
Map.addLayer(feature_selected_states,{'color':'808080'},"feature_selected_states")
Map.centerObject(feature_selected_states,8)

In [7]:
fire_start = ee.Date("2001-01-01")
fire_end   = ee.Date("2024-01-01")

# # for testing purpose, we set to 6 month and pahang only 
# fire_start = ee.Date("2015-01-01")
# fire_end   = ee.Date("2016-01-01")
# feature_selected_states = pahang_states #multipolygon
# selected_states = pahang_states #single polygon to reduce computation

# Historical Points Extraction 

## Fire Points

### MCd64A1 MODIS Burned Area Monthly Global 500m

In [8]:
dataset_MCD64A1 = ee.ImageCollection('MODIS/061/MCD64A1').select('BurnDate').filterDate(fire_start, fire_end)
dataset_MCD64A1 = dataset_MCD64A1.map(lambda image:image.clip(feature_selected_states))

burnedAreaVis = {
  'min': 0.0,
  'max': 366.0,
  'palette': ['4e0400', '951003', 'c61503', 'ff1901'],
}


Map.setCenter(102.206, 3.744, 7)
Map.addLayer(dataset_MCD64A1, burnedAreaVis, 'Burned Area')

### Sample Region - Extracting Fire Points from the MCD64A1 Area
- Extract all the coordinates points from the burnt area region
- Export the location as .csv - Yes (Actually is it necessary? We can just process the whole script, but will it crash?)
- Because we also need to find out the location of non-fire (pending) 

In [9]:
# loop through each of the image in image collection to get the points out 
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = feature_selected_states, # feature collection here
        scale = 1000, # Cell size of raster
        geometries = True
    )
    return feature


In [10]:
# modify this part to select more parameters to obtain from the image collection 
fire_points = dataset_MCD64A1.filterBounds(feature_selected_states).select('BurnDate').map(rasterExtraction).flatten()

In [11]:
# Add the label (fire = 1) indicating fire in the .csv 
def add_properties_fire(feature):
    # Get the geometry of the feature (point)
    point = ee.Geometry(feature.geometry())
    # Extract latitude and longitude from the point geometry
    latitude = point.coordinates().get(1)
    longitude = point.coordinates().get(0)
    properties = {'fire':1, 'latitude': latitude, 'longitude': longitude}
    return feature.set(properties)
fire_points = fire_points.map(add_properties_fire)

In [12]:
# fire_points

In [13]:
Map.addLayer(fire_points, {'color': 'purple'}, 'FireRegionPoints', 0)

In [14]:
# Map

## Non-Fire Point Extraction
- All available burnt area data within the region of interest from the MCD64A1 dataset is utilized.
- To improve the confidence of the the non-fire points,the FIRMS hotspots dataset is also included.
- The burnt areas and FIRMS hotspots are blended into a single image, depicting the historical burnt regions across all years. 
- An additional dilation morphological operation is applied to expand the boundaries of the burnt regions and hotspots, with the default radius and iteration value set to 2. 
- To obtain the non-fire regions, we invert the selection of burnt area region with the region of interest. 
- the GEE function ee.FeatureCollectionRandomPoints is employed to randomly extract non-fire points at a 1km² resolution, 
- with the total number of non-fire points matching the total number of fire points. 
- For the month and day of the fire for non-fire points, we can leverage the most recent available year in the MCD64A1 dataset. 

In [15]:
# Load 10 years firm dataset 
# dataset_firms = ee.ImageCollection('FIRMS').select('T21').filterDate(fire_start, fire_end)
dataset_firms = ee.ImageCollection('FIRMS').select('T21').filterDate(ee.Date("2014-01-01"), ee.Date("2024-01-01"))

# Clip dataset to selected area by using feature collection - ensure any hotspots outside of the selected area is not included
dataset_firms = dataset_firms.map(lambda image:image.clip(feature_selected_states))

# FIRMS_Visualization =  {'min': 1,
#                           'max': 100,
#                           'palette': ["white", "yellow", "orange", "red"]}


# # Add the firms to the maps
# Map.addLayer(dataset_firms, FIRMS_Visualization, '20 Years Accumulated Hotspot')

In [16]:
# Image collection --> Image with the total counts of fire hotspots
dataset_firms_count = dataset_firms.count() 

# Convert Fire Counts to binary (0 or 1) 
dataset_firms_binary = dataset_firms_count.eq(dataset_firms_count).rename('FIRMS_binary')

FIRMS_Visualization1 =  {'min': 0,
                          'max': 1,
                          'palette': ["green", "red"]}
Map.addLayer(dataset_firms_binary, FIRMS_Visualization1, 'firms_fire_binary',0)

In [17]:
# Take the burnt area dataset MCD64A1, combine with the firms dataset 
dataset_MCD64A1_count = dataset_MCD64A1.count()
dataset_MCD64A1_binary = dataset_MCD64A1_count.eq(dataset_MCD64A1_count).rename('FIRMS_binary')
Map.addLayer(dataset_MCD64A1_binary, FIRMS_Visualization1, 'dataset_MCD4A1_binary',0)

# .blend will overlays one image on top of another

merged_fire_binary = dataset_firms_binary.blend(dataset_MCD64A1_binary)
Map.addLayer(merged_fire_binary, FIRMS_Visualization1, 'merged_fire_binary',0)

In [18]:
# Perform a dilation with radius of 1 and iterations of 1 to increase the coverage area of FIRMS hotspots
# May change the iterations value to increase the size of firms hotspots area 
# Performing this to avoid the random points from taking any missed area of fire hotspots 
# By dilating, we can also reduce the number of features 
kernel = ee.Kernel.circle(**{'radius': 2})

# Perform a dilation, display.
merged_fire_binary = merged_fire_binary \
             .focalMax(**{'kernel': kernel, 'iterations': 2})

Map.addLayer(merged_fire_binary, FIRMS_Visualization1, 'merged_fire_dilation',0)


In [19]:
# Convert all the fire points to 0 
dataset_nofire = merged_fire_binary.eq(0)

# Create a mask by changing all the null values to -1 to show all the non-fire region 
dataset_nofire_mask = dataset_nofire.unmask(-1).clip(feature_selected_states)

# Update the mask by removing non fire region from the dataset 
dataset_nofire = dataset_nofire_mask.updateMask(dataset_nofire_mask.eq(-1))
Map.addLayer(dataset_nofire, FIRMS_Visualization1, 'firms_nofire',0)


In [20]:
# calling fire_points with more than 5000 points will result in unable to call
# may filter if want to call accordingly 
# fire_points.filter(ee.Filter.lt('BurnDate',50))

In [21]:
# Convert the binary firms to vectors 
# Using a too large scale will caused smaller pixels to be missed 
dataset_nofire_vector = dataset_nofire.reduceToVectors(
    **{
  'geometry': feature_selected_states,
  'scale': 1000,
  'geometryType': 'polygon',
  'eightConnected': False,})
Map.addLayer(dataset_nofire_vector.draw(**{'color': 'green', 'strokeWidth': 1}),{},'FIRMS No-Fire Area Vectorized', 0);

In [22]:
# Collect the same number points (as the number of firepoints) from the no_fire_vector
# Might take awile to complete the feature collection --> size (number) 
number_fire_points = fire_points.size()
nofire_points = ee.FeatureCollection.randomPoints(dataset_nofire_vector,number_fire_points)

# When the random points is sample, it might sample the boundaries of the feature_selected_states
# Thus, the filterBounds will be performed again to remove at the points of the boundary of the feature_selected_states
# Notes: 25/07/2023 - The codes will run into error if the points is at the boundary of the feature_selected_states 
nofire_points  = nofire_points.filterBounds(feature_selected_states)
Map.addLayer(nofire_points,{'color':'green', 'pointRadius': 1,'strokeWidth': 1},'No-Fire Random Points', 0)


In [23]:
def get_properties(feature):
    point = feature.geometry()
    
    # Find the properties from the property collection that correspond to the point
    properties = feature_selected_states.filterBounds(point).first().toDictionary()
    
    # Merge the existing properties of the point with the properties from the property collection
    return feature.set(properties)
    

# Add the label (fire = 0) indicating no fire in the .csv 
# Add the burndate as -1
# Add the latitude and longtiude 
def add_properties_nofire(feature):
    # Get the geometry of the feature (point)
    point = ee.Geometry(feature.geometry())
    # Extract latitude and longitude from the point geometry
    latitude = point.coordinates().get(1)
    longitude = point.coordinates().get(0)
    properties = {'fire':0, 'BurnDate':-1, 'latitude': latitude, 'longitude': longitude}
    return feature.set(properties)

nofire_points = nofire_points.map(get_properties)
nofire_points = nofire_points.map(add_properties_nofire)

In [24]:
# Map

## Merge Fire and No-Fire Points & Export as .csv
- combined both the points from fire and non fire
- use ee_to_csv to export the points to .csv file 
- recommended to export and save to avoid stressing GEE / crashing 
- easier to load back if anything crashes

In [25]:
#_____Merge Training Points______________________________-
allfire_combinedpoints = fire_points.merge(nofire_points)

In [26]:
# Export the fire & non-fire points to .csv in the same directory with name fire_dataset.csv 
# Might take some time depending on the number of points
geemap.ee_to_csv(allfire_combinedpoints, filename='fire_dataset.csv')

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\fire_dataset.csv


## Extract the Years / Months / Day from the Fire_dataset

In [27]:
df_fire_dataset = geemap.csv_to_df('fire_dataset.csv')

### Extract the Year Month and Day from system:index
### Extract day from BurnDate
### Replace non burn date with 31-12-2022

df_fire_dataset[['year', 'month']] = df_fire_dataset['system:index'].str.extract(r'_(\d{4})_(\d{2})_')
df_fire_dataset['year'].fillna('2023', inplace=True)
df_fire_dataset['month'].fillna('12', inplace=True)

# Replace -1 with 1 in the 'BurnDate' for 'day' column
df_fire_dataset['day'] = df_fire_dataset['BurnDate'].replace(-1, 31)
# Convert the day of year to the date of the month
df_fire_dataset['day'] = pd.to_datetime(df_fire_dataset['day'].astype(str) + ' ' + df_fire_dataset['year'].astype(str), format='%j %Y').dt.day

### Set Index
df_fire_dataset.set_index('system:index', inplace=True)

### Export fire_dataset
df_fire_dataset.to_csv("fire_dataset.csv")

# CSV Import back to Earth Engine

## Additional Module, Split to fire_points.csv and nofire_points.csv for Visualization in QGIS

In [28]:
df_fire_dataset_export = df_fire_dataset[df_fire_dataset['fire']==1]
df_nofire_dataset_export = df_fire_dataset[df_fire_dataset['fire']==0]

df_fire_dataset_export.to_csv('fire_points_only.csv')
df_nofire_dataset_export.to_csv('nofire_points_only.csv')

## Loading Fire Points Dataset 
- load fire points from the .csv back to gee 
- do not recommend to do all in one shot as GEE - out of memory / exceed resource
- Extract Year / Month / Day of Fire 


In [29]:
### Initialized ee , defined Map if haven't 
import ee
import geemap
import os
from geemap.datasets import DATA, get_metadata

# If the codes run from top, calling Map = geemap.Map() will remove all the previous layers 
try:
    Map
except NameError:
    print("Map not defined, define now!")
    Map = geemap.Map()
else:
    print("Map is defined, do not replace!")

Map is defined, do not replace!


In [30]:
import numpy as np
import pandas as pd
# import geopandas
from pandas import json_normalize
import json

In [31]:
### Load the fire_dataset.csv from the same directory as df 
df_fire_dataset = geemap.csv_to_df('fire_dataset.csv')

### Print out the number of fire points and non-fire points 
print("number of fire points =" , len(df_fire_dataset[df_fire_dataset['fire'] == 1]))
print("number of no_fire points =" , len(df_fire_dataset[df_fire_dataset['fire'] == 0]))

number of fire points = 5650
number of no_fire points = 5629


In [32]:
### Drop all columns containing NaN values as GEE cannot accept full NaN column 
df_fire_dataset = df_fire_dataset.dropna(axis=1, how='all')

In [33]:
### Filter the fire / no-fire points (some points) for smaller sets testing (use only 100 points)
### Comment the lines below if want to use the full set 
# df_fire_dataset = df_fire_dataset[df_fire_dataset['ADM1_PCODE']=="MY08"].sample(n=100)
# print("number of fire points =" , len(df_fire_dataset[df_fire_dataset['fire'] == 1]))
# print("number of no_fire points =" , len(df_fire_dataset[df_fire_dataset['fire'] == 0]))

In [34]:
### Convert pandaframes to EE feature collection
fire_dataset_points = geemap.pandas_to_ee(df_fire_dataset, latitude='latitude',longitude='longitude')

In [35]:
print("number of fire points =" , len(df_fire_dataset[df_fire_dataset['fire'] == 1]))
print("number of no_fire points =" , len(df_fire_dataset[df_fire_dataset['fire'] == 0]))

number of fire points = 5650
number of no_fire points = 5629


## Visualizing the Fire Points Dataset
- For sample visualization only (< 1000 points)
- use qgis if want to visualize all 10,000 points 

In [103]:
fire_dataset_points_map = fire_dataset_points.filter(ee.Filter.eq('fire', 1))
nofire_dataset_points_map = fire_dataset_points.filter(ee.Filter.eq('fire', 0))
Map.addLayer(fire_dataset_points_map,{'color':'red'}, "fire_dataset_points_map")
Map.addLayer(nofire_dataset_points_map,{'color':'green'}, "nofire_dataset_points_map")
# Map.addLayer(fire_dataset_points,{'color':'black'}, "fire_dataset.csv", 0)
Map.setCenter(102.206, 3.744, 7)
Map

## Remove Columns Before Extracting Variables from other Datasets 
- retain only important columns before extracting variables from GEE
- aka - remove all the states features and etc.
- reduce the number of features for each of the .csv files 

In [37]:
selected_column = ['latitude','longitude','fire','system:index','year','month','day']
df_fire_dataset_filtered = df_fire_dataset[selected_column]
fire_dataset_points = geemap.pandas_to_ee(df_fire_dataset_filtered, latitude='latitude',longitude='longitude')

# Pixels Extraction Notes
- Seperate into two section --> Image Collection / Image 
- For daily variables --> perform monthly aggregation in GEE to reduce the number of features extracted out to avoid resource/memory issues 
- For Monthly variables --> extract all of them, perform seasonal / annual average in pandas 
- For Annual variables --> extract all of them 
- For Monthly/Annual variables --> Create a new column "current_year_variables" (linking fire and variables year) 
- For each source, generate 1 csv files 
- However, due to GEE limitation, when too many points + variables, it will caused resource/memory issues
- Thus, extract each variables one by one, then merge all of them together 
- Once all the source have been extracted, merged the .csv using pandas (by matching the system:index with the original fire_dataset.csv) 
- The full dataset is now ready to be analysed 

## Declaring States and Fire start and end Variables
- Rerun to declare the feature collection of the area of interest in case GEE crashed

In [38]:
# Initialize variables before extracting
#Adding Malaysia States
all_states = ee.FeatureCollection("projects/chewyeejian/assets/mys_adm2")

#  _____Location Selection in Malaysia  via Feature Collection_____
#  [ADM1_PCODE] = MY01-Johor | MY02-Kedah | MY03-Kelantan | MY04-W.P. Kuala Lumpur | MY06-Melaka | MY07-Negeri | MY08-Pahang | MY09-Perak | MY10-Perlis | MY11-Pulau Pinang | MY15-Terengganu | MY16-W.P. Putrajaya | MY17-Selangor
#  [ADM1_PCODE] = MY05-W.P. Labuan | MY12-Sabah | MY13-Sarawak
peninsular = all_states.filter(ee.Filter.Or(
                                    ee.Filter.eq('ADM1_PCODE',"MY01"),ee.Filter.eq('ADM1_PCODE',"MY02"),ee.Filter.eq('ADM1_PCODE',"MY03"),
                                    ee.Filter.eq('ADM1_PCODE',"MY04"),ee.Filter.eq('ADM1_PCODE',"MY06"),ee.Filter.eq('ADM1_PCODE',"MY07"),
                                    ee.Filter.eq('ADM1_PCODE',"MY08"),ee.Filter.eq('ADM1_PCODE',"MY09"),ee.Filter.eq('ADM1_PCODE',"MY10"),                                    
                                    ee.Filter.eq('ADM1_PCODE',"MY11"),ee.Filter.eq('ADM1_PCODE',"MY14"),ee.Filter.eq('ADM1_PCODE',"MY15"),
                                    ee.Filter.eq('ADM1_PCODE',"MY16"),ee.Filter.eq('ADM1_PCODE',"MY17")                                    
                                    ))

sabahsarawak_states = all_states.filter(ee.Filter.Or(
                                    ee.Filter.eq('ADM1_PCODE',"MY05"),ee.Filter.eq('ADM1_PCODE',"MY13"),ee.Filter.eq('ADM1_PCODE',"MY12")))

pahang_states = all_states.filter(ee.Filter.Or
                                  (ee.Filter.eq('ADM1_PCODE',"MY08")))


#  _____Location Selection in Malaysia  via Feature Collection_____
feature_selected_states = peninsular #multipolygon
selected_states = peninsular #single polygon to reduce computation
# print(feature_selected_states,'feature_selected_states')
Map.addLayer(feature_selected_states,{'color':'808080'},"feature_selected_states")
Map.centerObject(feature_selected_states,8)


fire_start = ee.Date("2001-01-01")
fire_end   = ee.Date("2024-01-01")

# # for testing purpose, we set to 6 month and pahang only 
# fire_start = ee.Date("2021-01-01")
# fire_end   = ee.Date("2023-01-01")
# feature_selected_states = pahang_states #multipolygon
# selected_states = pahang_states #single polygon to reduce computation

# Image Collection

## Use pandas to extract yearly and seasonal averages
- why we do this? Instead of doing in GEE, extract everything
- allow for better data analysis / data scientist to easily work with it
- however, when the number of bands is more than 5000, GEE will not allow us to extract
- hence, if daily information is provided, we shall extract the monthly average before export 

In [39]:
### Function to process the column name to obtain the annual / seasonal values from the current "date" of fire  
def process_annual_seasonal(df_ori):
    ### Set the index to system:index
    df_ori.set_index('system:index', inplace=True)
    
    ### Make a copy of the original df
    df = df_ori.copy()
    
    ### obtain the variable names through the columns names
    column_list = df.columns[df.columns.str.contains('_')].str.split('_').str[1].drop_duplicates()  
    print(column_list)
    
    ### loops to loop through each of the variables to obtain the seasonal values and annual average 
    ## ensure that the variable format is YYYYMM_????
    ## annual --> after split with _, select the first element, and take only the first 4 character which is the year
    ## seasonal average
    for items in column_list:
        print("calculating annual average and seasonal average for "+items)
        df = df.filter(like=items).groupby(by=lambda x: str(x.split('_')[0][:4]), axis=1).mean().add_suffix('_'+items+'_annual').merge(df, on='system:index', how='outer')
        df = df.filter(regex=r'^\d{4}(12|01|02)_'+items).groupby(by=lambda x: str(x.split('_')[0][:4]), axis=1).mean().add_suffix('_'+items+'_DJF').merge(df, on='system:index', how='outer')
        df = df.filter(regex=r'^\d{4}(03|04|05)_'+items).groupby(by=lambda x: str(x.split('_')[0][:4]), axis=1).mean().add_suffix('_'+items+'_MAM').merge(df, on='system:index', how='outer')
        df = df.filter(regex=r'^\d{4}(06|07|08)_'+items).groupby(by=lambda x: str(x.split('_')[0][:4]), axis=1).mean().add_suffix('_'+items+'_JJA').merge(df, on='system:index', how='outer')
        df = df.filter(regex=r'^\d{4}(09|10|11)_'+items).groupby(by=lambda x: str(x.split('_')[0][:4]), axis=1).mean().add_suffix('_'+items+'_SON').merge(df, on='system:index', how='outer')
        
    ### Combined the annual/seasonal average with the original df      
    df_ori = df_ori.combine_first(df)
#         print(df_ori)

    ### Create another copy of the df containing the annual/seasonal average 
    df = df_ori.copy()
    
    ### Loop through each of the available year in [year] column 
    ##  Find the current year variables and rename the column name to current 
    ##  Compare the year in the "year column" with the variables (column) name YYYYMM 
    ##  Combine the extracted columns with the original df 
    print("obtaining the current year average for each of the fire point ")
    for year in df_ori['year'].unique():
#         print(year)
        df = df_ori.copy()
        regex_pattern = r'^' + str(year)     
        df = df.query('year == @year').filter(regex=regex_pattern)
        df.columns = df.columns.str.replace(r'^\d{4}', 'current', regex=True) 
        df_ori = df_ori.combine_first(df)
        
    
    ## beautify some of the curren (e.g., current01_?? --> current_01_??)
    ## not working 11/09/2023
#     df_ori.columns = df_ori.columns.str.replace(r'current\d+', 'current_', regex=True)
#     df.filter(regex=r'current\d+').columns.str.replace('current','current_')
#     df.columns.str.contains('current\d+',regex=True)

    return df_ori            

In [40]:
### Function to process the column name to obtain the current year variables 
## This function is a replicate of the previous, however, it is used for annual variables to reduce computation 
def process_current_year(df_ori):
    ### Set the index to system:index
    df_ori.set_index('system:index', inplace=True)

    ### standardized naming - add suffix annual to all landcover
    df = df_ori.filter(regex=r'\d{4}').add_suffix('_annual')                     #filter all landcover and add suffix 
    df_ori = df_ori[df_ori.columns.drop(list(df_ori.filter(regex=r'\d{4}')))]    #drop all landcover previously have the year
    df_ori = df_ori.combine_first(df)                                            #combine back with the one added with suffix 

    print("obtaining the current year average for each of the fire point ")
    for year in df_ori['year'].unique():
#         print(year)
        df = df_ori.copy()
        regex_pattern = r'^' + str(year)     
        df = df.query('year == @year').filter(regex=regex_pattern)
        df.columns = df.columns.str.replace(r'^\d{4}', 'current', regex=True) 
        df_ori = df_ori.combine_first(df)
        
    return df_ori            

## Monthly Average in Google Earth Engine
- obtain the monthly average from google earth engine
- GEE doesn't allow you to export more than 5000 bands 
- Reason to extract all the monthly data - easier for data analysis work 
- Format for KBDI 202201_KBDI 
- Format for Landsat temperature 2022_01_LST (need to moreve the _ from system index )

In [41]:
def monthly_Avg (collection, years, months):
    avg = []
    for year in years:
        for month in months:
            #Get the system index from the first collection of every month, get only the first 6 character 
            print (year, month)
#             system_index   = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
#                                   .filter(ee.Filter.calendarRange(month, month, 'month')) \
#                                   .first().getInfo().get('properties').get('system:index')[:6].replace('_','')
            # instead of using the system index directly, we map the year + month (in two digit) to generalize
            system_index = (str(year) + "{:02d}".format(month))
            
            #Get the monthly average 
            Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                  .filter(ee.Filter.calendarRange(month, month, 'month')) \
                                  .mean() \
                                  .set({'month': month, 'year': year, 'system:index': system_index})
            avg.append (Monthly_avg)
    return ee.ImageCollection.fromImages(avg)


## Terra Climate
- monthly data
- 1958-01-01 to 2022-12-01
- 4km (native scale) 
- seasonal / annual - in pandas 

In [42]:
### Load the dataset and clip to the selected area 
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
                  .filter(ee.Filter.date(fire_start, fire_end))
terraclimate = terraclimate.map(lambda image:image.clip(feature_selected_states))

In [43]:
### Applying Scale Factors
### .map cannot use client side loops, this is currently the best way to apply all 
def apply_scale(image):
    aet_s = image.select(['aet']).multiply(0.1)
    def_s = image.select(['def']).multiply(0.1)
    pdsi_s = image.select(['pdsi']).multiply(0.01)
    pet_s = image.select(['pet']).multiply(0.1)
#     pr_s = image.select(['pr']) # scale=1.0
#     ro_s = image.select(['ro']) # scale=1.0
    soil_s = image.select(['soil']).multiply(0.1)
    srad_s = image.select(['srad']).multiply(0.1)
#     swe_s = image.select(['swe'])  # scale=1.0
    tmmn_s = image.select(['tmmn']).multiply(0.1)
    tmmx_s = image.select(['tmmx']).multiply(0.1)
    vap_s = image.select(['vap']).multiply(0.001)
    vpd_s = image.select(['vpd']).multiply(0.01)
    vs_s = image.select(['vs']).multiply(0.01)
    return image.addBands([aet_s, def_s, pdsi_s, pet_s, soil_s, srad_s, tmmn_s, tmmx_s, vap_s, vpd_s, vs_s], overwrite=True)

terraclimate = terraclimate.map(apply_scale)

In [ ]:
# ### if not too many points / date range, can directly use this (extracted one by one due to computation resource error gee)
# ### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
# ### image collection cannot use extract_values_to_points
# terraclimate = terraclimate.toBands()
# work_dir = os.path.expanduser('')
# out_csv = os.path.join(work_dir, 'terraclimate.csv')
# geemap.extract_values_to_points(fire_dataset_points, terraclimate, out_csv, geometries=True, scale=1000)

In [44]:
### Extract variable one by one due to memory exceeded / resource exceeded 
### not recommended to use looping as the loop might break due to reliance on GEE 
selected = terraclimate.select('aet')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'aet.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('def')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'def.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('pdsi')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'pdsi.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\aet.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\def.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\pdsi.csv


In [45]:
selected = terraclimate.select('pet')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'pet.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('pr')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'pr.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('ro')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'ro.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\pet.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\pr.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\ro.csv


In [46]:
selected = terraclimate.select('soil')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'soil.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('srad')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'srad.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('swe')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'swe.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\soil.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\srad.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\swe.csv


In [47]:
selected = terraclimate.select('tmmn')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'tmmn.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('tmmx')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'tmmx.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('vap')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'vap.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\tmmn.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\tmmx.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\vap.csv


In [48]:
selected = terraclimate.select('vpd')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'vpd.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

selected = terraclimate.select('vs')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'vs.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\vpd.csv
Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\vs.csv


In [49]:
### Combine all the variables .csv together 
aet_df = pd.read_csv('aet.csv')
def_df = pd.read_csv('def.csv')
pdsi_df = pd.read_csv('pdsi.csv')
pet_df = pd.read_csv('pet.csv')
pr_df = pd.read_csv('pr.csv')
ro_df = pd.read_csv('ro.csv')
soil_df = pd.read_csv('soil.csv')
srad_df = pd.read_csv('srad.csv')
swe_df = pd.read_csv('swe.csv')
tmmn_df = pd.read_csv('tmmn.csv')
tmmx_df = pd.read_csv('tmmx.csv')
vap_df = pd.read_csv('vap.csv')
vpd_df = pd.read_csv('vpd.csv')
vs_df = pd.read_csv('vs.csv')


### Initialize the merged_df DataFrame
merged_df = aet_df

# Join each DataFrame with merged_df and  then duplicate column name will have suffix of DROP, filter away any column that contain DROP 
merged_df = merged_df.join(def_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(pdsi_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(pet_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(pr_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(ro_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(soil_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(srad_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(swe_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(tmmn_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(tmmx_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(vap_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(vpd_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(vs_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")


# Save the combined DataFrame to a new CSV file
merged_df.set_index('system:index', inplace=True)
merged_df.to_csv('terraclimate.csv', index=True)

In [50]:
merged_df.to_csv('terraclimate.csv', index=True)

In [52]:
### Load .csv - obtain annual/sesaonl average & current year values 
df = geemap.csv_to_df('terraclimate.csv')
df = process_annual_seasonal(df)
df.to_csv("terraclimate.csv")

Index(['aet', 'def', 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn',
       'tmmx', 'vap', 'vpd', 'vs'],
      dtype='object')
calculating annual average and seasonal average for aet
calculating annual average and seasonal average for def
calculating annual average and seasonal average for pdsi
calculating annual average and seasonal average for pet
calculating annual average and seasonal average for pr
calculating annual average and seasonal average for ro
calculating annual average and seasonal average for soil
calculating annual average and seasonal average for srad
calculating annual average and seasonal average for swe
calculating annual average and seasonal average for tmmn
calculating annual average and seasonal average for tmmx
calculating annual average and seasonal average for vap
calculating annual average and seasonal average for vpd
calculating annual average and seasonal average for vs
obtaining the current year average for each of the fire point 


## KBDI
- 2007-01-01T00:00:00Z–2023-07-26T00:00:00
- daily
- 4000 meters 
- consider mean monthly, annual, seasonal 

In [53]:
KBDI = ee.ImageCollection('UTOKYO/WTLAB/KBDI/v1') \
  .select('KBDI') \
  .filterDate(fire_start, fire_end)
KBDI = KBDI.map(lambda image:image.clip(feature_selected_states))
bandViz = {
  'min': 0,
  'max': 800,
  'palette': [
    '001a4d', '003cb3', '80aaff', '336600', 'cccc00', 'cc9900', 'cc6600',
    '660033'
  ]
}
Map.addLayer(KBDI.mean(), bandViz, 'Keetch-Byram Drought Index')

In [54]:
# Define the years and months range for extracting annual average and monthly average 
years = range(fire_start.get('year').getInfo(), fire_end.get('year').getInfo())
# Manually change the month if error, or it is not starting from january 
months = range(1,13)
KBDI = monthly_Avg(KBDI,years,months)

2001 1
2001 2
2001 3
2001 4
2001 5
2001 6
2001 7
2001 8
2001 9
2001 10
2001 11
2001 12
2002 1
2002 2
2002 3
2002 4
2002 5
2002 6
2002 7
2002 8
2002 9
2002 10
2002 11
2002 12
2003 1
2003 2
2003 3
2003 4
2003 5
2003 6
2003 7
2003 8
2003 9
2003 10
2003 11
2003 12
2004 1
2004 2
2004 3
2004 4
2004 5
2004 6
2004 7
2004 8
2004 9
2004 10
2004 11
2004 12
2005 1
2005 2
2005 3
2005 4
2005 5
2005 6
2005 7
2005 8
2005 9
2005 10
2005 11
2005 12
2006 1
2006 2
2006 3
2006 4
2006 5
2006 6
2006 7
2006 8
2006 9
2006 10
2006 11
2006 12
2007 1
2007 2
2007 3
2007 4
2007 5
2007 6
2007 7
2007 8
2007 9
2007 10
2007 11
2007 12
2008 1
2008 2
2008 3
2008 4
2008 5
2008 6
2008 7
2008 8
2008 9
2008 10
2008 11
2008 12
2009 1
2009 2
2009 3
2009 4
2009 5
2009 6
2009 7
2009 8
2009 9
2009 10
2009 11
2009 12
2010 1
2010 2
2010 3
2010 4
2010 5
2010 6
2010 7
2010 8
2010 9
2010 10
2010 11
2010 12
2011 1
2011 2
2011 3
2011 4
2011 5
2011 6
2011 7
2011 8
2011 9
2011 10
2011 11
2011 12
2012 1
2012 2
2012 3
2012 4
2012 5
2012 6
2

In [55]:
### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
### image collection cannot use extract_values_to_points
KBDI = KBDI.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'KBDI.csv')
### Scale Parameters are required else will come out error due to the monthly mean and etc. 
###  EEException: Image.reduceRegions: The default WGS84 projection is invalid for aggregations. Specify a scale or crs & crs_transform.
geemap.extract_values_to_points(fire_dataset_points, KBDI, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\KBDI.csv


In [56]:
### Annual/ Seaonal/ Monthly Average of the current year of fire  
df = geemap.csv_to_df('KBDI.csv')
# set index to system index 
df = process_annual_seasonal(df)
df.to_csv("KBDI.csv")

Index(['KBDI'], dtype='object')
calculating annual average and seasonal average for KBDI
obtaining the current year average for each of the fire point 


## MOD11A2.061 Terra Land Surface Temperature and Emissivity 8-Day Global 1km
- Dataset Availability
- 2000-02-18T00:00:00Z–2023-07-04T00:00:00
- 1000 meters
- every 8 days
- consider mean monthly, annual, seasonal

In [57]:
landsurfacetemperature = ee.ImageCollection('MODIS/061/MOD11A2') \
                  .filter(ee.Filter.date(fire_start, fire_end))
landsurfacetemperature = landsurfacetemperature.map(lambda image:image.clip(feature_selected_states))
landsurfacetemperature = landsurfacetemperature.select('LST_Day_1km')
landsurfacetemperatureVis = {
  'min': 14000.0,
  'max': 16000.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
}
Map.addLayer(
    landsurfacetemperature, landsurfacetemperatureVis,
    'Land Surface Temperature')

In [58]:
### Applying Scale Factors
### .map cannot use client side loops, this is currently the best way to apply all 
def apply_scale(image):
    lst_s = image.select(['LST_Day_1km']).multiply(0.02)
    return image.addBands([lst_s], overwrite=True)

landsurfacetemperature = landsurfacetemperature.map(apply_scale)

In [59]:
# Define the years and months range for extracting annual average and monthly average 
years = range(fire_start.get('year').getInfo(), fire_end.get('year').getInfo())

# Manually change the month if error, or it is not starting from january 
months = range(1,13)

### Obtain the monthly average values
landsurfacetemperature = monthly_Avg(landsurfacetemperature,years,months)

2001 1
2001 2
2001 3
2001 4
2001 5
2001 6
2001 7
2001 8
2001 9
2001 10
2001 11
2001 12
2002 1
2002 2
2002 3
2002 4
2002 5
2002 6
2002 7
2002 8
2002 9
2002 10
2002 11
2002 12
2003 1
2003 2
2003 3
2003 4
2003 5
2003 6
2003 7
2003 8
2003 9
2003 10
2003 11
2003 12
2004 1
2004 2
2004 3
2004 4
2004 5
2004 6
2004 7
2004 8
2004 9
2004 10
2004 11
2004 12
2005 1
2005 2
2005 3
2005 4
2005 5
2005 6
2005 7
2005 8
2005 9
2005 10
2005 11
2005 12
2006 1
2006 2
2006 3
2006 4
2006 5
2006 6
2006 7
2006 8
2006 9
2006 10
2006 11
2006 12
2007 1
2007 2
2007 3
2007 4
2007 5
2007 6
2007 7
2007 8
2007 9
2007 10
2007 11
2007 12
2008 1
2008 2
2008 3
2008 4
2008 5
2008 6
2008 7
2008 8
2008 9
2008 10
2008 11
2008 12
2009 1
2009 2
2009 3
2009 4
2009 5
2009 6
2009 7
2009 8
2009 9
2009 10
2009 11
2009 12
2010 1
2010 2
2010 3
2010 4
2010 5
2010 6
2010 7
2010 8
2010 9
2010 10
2010 11
2010 12
2011 1
2011 2
2011 3
2011 4
2011 5
2011 6
2011 7
2011 8
2011 9
2011 10
2011 11
2011 12
2012 1
2012 2
2012 3
2012 4
2012 5
2012 6
2

In [60]:
### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
### image collection cannot use extract_values_to_points
landsurfacetemperature = landsurfacetemperature.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'landsurfacetemperature.csv')
geemap.extract_values_to_points(fire_dataset_points, landsurfacetemperature, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\landsurfacetemperature.csv


In [61]:
### Annual/ Seaonal/ Monthly Average of the current year of fire  
df = geemap.csv_to_df('landsurfacetemperature.csv')
# set index to system index 
df = process_annual_seasonal(df)
df.to_csv("landsurfacetemperature.csv")

Index(['LST'], dtype='object')
calculating annual average and seasonal average for LST
obtaining the current year average for each of the fire point 


## NDVI MOD13Q1.061 Terra 
- Dataset Availability
2000-02-18T00:00:00Z–2023-06-26T00:00:00
- 16 days 
- 250m

In [62]:
ndvi_evi = ee.ImageCollection('MODIS/061/MOD13Q1') \
                  .filter(ee.Filter.date(fire_start, fire_end))
ndvi_evi = ndvi_evi.map(lambda image:image.clip(feature_selected_states))
ndvi_evi = ndvi_evi.select('NDVI','EVI')

# # Visualize
# ndvi = dataset.select('NDVI')
# ndviVis = {
#   'min': 0.0,
#   'max': 8000.0,
#   'palette': [
#     'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
#     '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
#     '012E01', '011D01', '011301'
#   ],
# }
# Map.addLayer(ndvi, ndviVis, 'NDVI')

In [63]:
### Applying Scale Factors
### .map cannot use client side loops, this is currently the best way to apply all 
def apply_scale(image):
    ndvi_s = image.select(['NDVI']).multiply(0.0001)
    evi_s = image.select(['EVI']).multiply(0.0001)
    return image.addBands([ndvi_s,evi_s], overwrite=True)

ndvi_evi = ndvi_evi.map(apply_scale)

In [64]:
### Obtain the monthly average values
ndvi_evi = monthly_Avg(ndvi_evi,years,months)

2001 1
2001 2
2001 3
2001 4
2001 5
2001 6
2001 7
2001 8
2001 9
2001 10
2001 11
2001 12
2002 1
2002 2
2002 3
2002 4
2002 5
2002 6
2002 7
2002 8
2002 9
2002 10
2002 11
2002 12
2003 1
2003 2
2003 3
2003 4
2003 5
2003 6
2003 7
2003 8
2003 9
2003 10
2003 11
2003 12
2004 1
2004 2
2004 3
2004 4
2004 5
2004 6
2004 7
2004 8
2004 9
2004 10
2004 11
2004 12
2005 1
2005 2
2005 3
2005 4
2005 5
2005 6
2005 7
2005 8
2005 9
2005 10
2005 11
2005 12
2006 1
2006 2
2006 3
2006 4
2006 5
2006 6
2006 7
2006 8
2006 9
2006 10
2006 11
2006 12
2007 1
2007 2
2007 3
2007 4
2007 5
2007 6
2007 7
2007 8
2007 9
2007 10
2007 11
2007 12
2008 1
2008 2
2008 3
2008 4
2008 5
2008 6
2008 7
2008 8
2008 9
2008 10
2008 11
2008 12
2009 1
2009 2
2009 3
2009 4
2009 5
2009 6
2009 7
2009 8
2009 9
2009 10
2009 11
2009 12
2010 1
2010 2
2010 3
2010 4
2010 5
2010 6
2010 7
2010 8
2010 9
2010 10
2010 11
2010 12
2011 1
2011 2
2011 3
2011 4
2011 5
2011 6
2011 7
2011 8
2011 9
2011 10
2011 11
2011 12
2012 1
2012 2
2012 3
2012 4
2012 5
2012 6
2

In [ ]:
# ### Cannot extract all of them together - memory exceeded and resource exceeded 
# ### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
# ### image collection cannot use extract_values_to_points
# ndvi_evi = ndvi_evi.toBands()
# work_dir = os.path.expanduser('')
# out_csv = os.path.join(work_dir, 'ndvi_evi.csv')
# geemap.extract_values_to_points(fire_dataset_points, ndvi_evi, out_csv, geometries=True, scale=1000)

In [65]:
### Extract variable one by one due to memory exceeded / resource exceeded 
### not recommended to use looping as the loop might break due to reliance on GEE 
selected = ndvi_evi.select('NDVI')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'ndvi.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\ndvi.csv


In [66]:
selected = ndvi_evi.select('EVI')
selected = selected.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'evi.csv')
geemap.extract_values_to_points(fire_dataset_points, selected, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\evi.csv


In [67]:
### Combine all the variables .csv together 
ndvi_df = pd.read_csv('ndvi.csv')
evi_df = pd.read_csv('evi.csv')

## join the csv files, then duplicate column name will have suffix of DROP, filter away any column that contain DROP 
merged_df = ndvi_df.join(evi_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")

# Save the combined DataFrame to a new CSV file
merged_df.set_index('system:index', inplace=True)
merged_df.to_csv('ndvi_evi.csv', index=True)

In [68]:
### Annual/ Seaonal/ Monthly Average of the current year of fire  
df = geemap.csv_to_df('ndvi_evi.csv')
# set index to system index 
df = process_annual_seasonal(df)
df.to_csv("ndvi_evi.csv")

Index(['NDVI', 'EVI'], dtype='object')
calculating annual average and seasonal average for NDVI
calculating annual average and seasonal average for EVI
obtaining the current year average for each of the fire point 


## MODIS MCD12Q1 Landcover
- 2001-2021
- annual / yearly
- 500m
- task: read on the bands Land covert ype by different organization and etc. before deciding? 


In [69]:
MCD12Q1 = ee.ImageCollection('MODIS/061/MCD12Q1') \
          .filterDate(fire_start, fire_end)
MCD12Q1 = MCD12Q1.map(lambda image:image.clip(feature_selected_states))
MCD12Q1 = MCD12Q1.select('LC_Type2')

MCD12Q1Vis = {
  'min': 0.0,
  'max': 15.0,
  'palette': [
        '1c0dff', '05450a', '086a10', '54a708', '78d203', 
        '009900', 'c6b044', 'dcd159', 'dade48', 'fbff13', 
        'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', 'f9ffa4'
  ],
}
Map.addLayer(MCD12Q1, MCD12Q1Vis, 'MCD12Q1 UMD land cover ')

In [70]:
### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
### image collection cannot use extract_values_to_points
MCD12Q1 = MCD12Q1.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'MCD12Q1.csv')
geemap.extract_values_to_points(fire_dataset_points, MCD12Q1, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\MCD12Q1.csv


In [71]:
### Annual/ Seaonal/ Monthly Average of the current year of fire  
df = geemap.csv_to_df('MCD12Q1.csv')


### standardized the naming as other variables  (YYYYMM_VariableName)
df.columns = df.columns.str.replace('_','',2)
# df.columns = df.columns.str.replace('-','')

### Annual/ Seaonal/ Monthly Average of the current year of fire 
### Although no seasonal/monthly, it will extract the current year 
# df = process_annual_seasonal(df)
df = process_current_year(df)

### Process landcover dataset 

# Define the mapping of values and class 
mcd12q1_class_mapping = {
    0: 'Water Bodies',
    1: 'Evergreen Needleleaf Forests',
    2: 'Evergreen Broadleaf Forests',
    3: 'Deciduous Needleleaf Forests',
    4: 'Deciduous Broadleaf Forests',
    5: 'Mixed Forests',
    6: 'Closed Shrublands',
    7: 'Open Shrublands',
    8: 'Woody Savannas',
    9: 'Savannas',
    10: 'Grasslands',
    11: 'Permanent Wetlands',
    12: 'Croplands',
    13: 'Urban and Built-up Lands',
    14: 'Cropland/Natural Vegetation Mosaics',
    15: 'Non-Vegetated Lands'
}

# Use the replace() function to populate classname
# Merge with the original dataframe 
df_classname = df.filter(regex='.*LC_').replace(mcd12q1_class_mapping).add_suffix("_classname")
df = df.merge(df_classname, on='system:index', how='outer')

df.to_csv("MCD12Q1.csv")

obtaining the current year average for each of the fire point 


In [72]:
df = pd.read_csv('MCD12Q1.csv')
### For non-fire points, use the latest year available
df.loc[df['fire'] == 0, 'current0101_LC_Type2_annual'] = df['20220101_LC_Type2_annual']
df.loc[df['fire'] == 0, 'current0101_LC_Type2_annual_classname'] = df['20220101_LC_Type2_annual_classname']
df.to_csv("MCD12Q1.csv")

## Human Footprint
- https://wcshumanfootprint.org/data-access
- 300m
- 2001 – 2020 
- The footprint is a simple weighted sum of maps of where people live (population density), where we build infrastructure (including roads, railways, factories, and other kinds of infrastructure), where we can go (accessibility), and where we use electrical energy, a proxy for access to industrial energy supplies, as measured by the night-time lights.
- Just take the human foot print
- Contain other attribute as well : 
- Infrastructure, Landuse	Population density, Power,	Railway, Roads,	Water
- Consider in the future for anthropogenic analysis (calculating distance from the fire location) 

In [73]:
human_impact_index = ee.ImageCollection("projects/HII/v1/hii") \
                        .filter(ee.Filter.date(fire_start, fire_end))
human_impact_index = human_impact_index.map(lambda image:image.clip(feature_selected_states))
# hiiviz = {'min': 5, 'max': 5000, 'palette': ["224f1a","a3ff76","feff6f","a09568","ffa802","f7797c","fb0102","d87136","a90086","7a1ca5","421137","000000"]}
# Map.addLayer(human_impact_index, hiiviz, "Human Impact Index")


In [74]:
### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
### image collection cannot use extract_values_to_points
human_impact_index = human_impact_index.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'hii.csv')
geemap.extract_values_to_points(fire_dataset_points, human_impact_index, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\hii.csv


In [75]:
### Annual/ Seaonal/ Monthly Average of the current year of fire  
df = geemap.csv_to_df('hii.csv')

### standardized the naming as other variables  (YYYYMM_VariableName)
df.columns = df.columns.str.replace('hii_','')
df.columns = df.columns.str.replace('-','')

### Although no seasonal/monthly, it will extract the current year 
df = process_current_year(df)

obtaining the current year average for each of the fire point 


In [76]:
### For non-fire points, use the latest year available
df.loc[df['fire'] == 0, 'current0101_hii_annual'] = df['20200101_hii_annual']

In [77]:
df.to_csv("hii.csv")

## VIIRS Nighttime Day/Night Annual Band Composites (V2.1)
- annual
- 2012-2021
- Cloud free 
- 500m 


In [78]:
nighttime = ee.ImageCollection('NOAA/VIIRS/DNB/ANNUAL_V21') \
                .filter(ee.Filter.date(fire_start, fire_end))
nighttime = nighttime.map(lambda image:image.clip(feature_selected_states))
nighttime = nighttime.select('average')
Map.addLayer(nighttime, {}, 'NOAA/VIIRS/DNB/ANNUAL_V21')

In [79]:
### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
### image collection cannot use extract_values_to_points
nighttime = nighttime.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'nighttime1.csv')
geemap.extract_values_to_points(fire_dataset_points, nighttime, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\nighttime1.csv


### VIIRS Nighttime Day/Night Annual Band Composites (V2.2)
- https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_ANNUAL_V22
- 2022 onwards

In [80]:
nighttime = ee.ImageCollection('NOAA/VIIRS/DNB/ANNUAL_V22') \
                .filter(ee.Filter.date(fire_start, fire_end))
nighttime = nighttime.map(lambda image:image.clip(feature_selected_states))
nighttime = nighttime.select('average')
Map.addLayer(nighttime, {}, 'NOAA/VIIRS/DNB/ANNUAL_V22')

In [81]:
### toBands -convert a collection multi-band image containing all of the bands of every image in the collection 
### image collection cannot use extract_values_to_points
nighttime = nighttime.toBands()
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'nighttime2.csv')
geemap.extract_values_to_points(fire_dataset_points, nighttime, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\nighttime2.csv


In [82]:
### Because the V2.2 only contain single image, the naming is different
### To standardized, we updated the column 'first' to '20220101_average'

df = geemap.csv_to_df('nighttime2.csv')
df.rename(columns={'first': '20220101_average'}, inplace=True)
df.set_index('system:index', inplace=True)
df.to_csv("nighttime2.csv")

### Combine V2.1 and V2.2, extract Current Year 

In [83]:
### Combine the nighttime from V2.1 and V2.2
nighttime_df1 = pd.read_csv('nighttime1.csv')
nighttime_df2 = pd.read_csv('nighttime2.csv')

### join the csv files, then duplicate column name will have suffix of DROP, filter away any column that contain DROP 
merged_df = nighttime_df1.join(nighttime_df2, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df.set_index('system:index', inplace=True)
merged_df.to_csv("nighttime.csv")

In [84]:
### Process dataset / rename column name
df = geemap.csv_to_df('nighttime.csv')

### Although no seasonal/monthly, it will extract the current year 
df = process_current_year(df)

# rename the column 
# add suffix to the original data
# drop the original data 
# Merge with the original dataframe 
df_classname = df.filter(regex='.*average').add_suffix("_nighttime")
df = df[df.columns.drop(list(df.filter(regex='.*average')))]
df = df.merge(df_classname, on='system:index', how='outer')

obtaining the current year average for each of the fire point 


In [85]:
### For non-fire points, use the latest year available
df.loc[df['fire'] == 0, 'current0101_average_annual_nighttime'] = df['20220101_average_annual_nighttime']

In [86]:
df.to_csv("nighttime.csv")

# Image

## ESA landcover - Image
- refer https://worldcover2020.esa.int/data/docs/WorldCover_PUM_V1.1.pdf for details related to the label 
- 2021-01-01 to 2022-01-01
- Annual	
- Landcover (Map)
- static

In [87]:
esa = ee.ImageCollection('ESA/WorldCover/v200')
esa = esa.map(lambda image:image.clip(feature_selected_states))
esa = esa.first()
# dataset_landcover = ee.Filter.bounds(feature_selected_states)
visualization = {
  'bands': ['Map'],
}

# Map.centerObject(dataset_landcover)
# Map.addLayer(esa, visualization, "esa")

In [88]:
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'esa.csv')
geemap.extract_values_to_points(fire_dataset_points, esa, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\esa.csv


In [89]:
### Process esa landcover dataset 
df = geemap.csv_to_df('esa.csv')
df.set_index('system:index', inplace=True)

# rename the column first to esa_class_values
df = df.rename(columns={'first':'esa_class_values'})

# Define the mapping of esa_class_values to esa_class_name
esa_class_mapping = {
    10: 'Tree cover',
    20: 'Shrubland',
    30: 'Grassland',
    40: 'Cropland',
    50: 'Built-up',
    60: 'Bare / sparse vegetation',
    70: 'Snow and ice',
    80: 'Permanent water bodies',
    90: 'Herbaceous wetland',
    95: 'Mangroves',
    100: 'Moss and lichen'
}

# Use the replace() function to populate esa_class_name
df['esa_class_name'] = df['esa_class_values'].replace(esa_class_mapping)
df.to_csv("esa.csv")

## NASADEM
- NASADEM  [22]
- (static) 		
- 2000-02-11 to 2000-02-22	
- 30m


In [90]:

dataset_dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
dataset_dem = dataset_dem.clip(feature_selected_states)

### Calculate slope. Units are degrees, range is [0,90).
# dataset_slope = ee.Terrain.slope(dataset_dem)

### Calculate aspect. Units are degrees where 0=N, 90=E, 180=S, 270=W.
# dataset_aspect = ee.Terrain.aspect(dataset_dem)

### Use the ee.Terrain.products function to calculate slope, aspect, and
### hillshade simultaneously. The output bands are appended to the input image.
### Hillshade is calculated based on illumination azimuth=270, elevation=45.
dataset_terrain = ee.Terrain.products(dataset_dem)
# print('ee.Terrain.products bands', terrain.bandNames())

### Display slope and aspect and terrain layers on the map.
# Map.addLayer(dataset_slope, {'min': 0, 'max': 89.99}, 'Slope')
# Map.addLayer(dataset_aspect, {'min': 0, 'max': 359.99}, 'Aspect')
Map.addLayer(dataset_terrain.select('hillshade'), {'min': 0, 'max': 255}, 'Hillshade')
# Map.setCenter(102.206, 3.744, 7)


In [91]:
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'dem.csv')
geemap.extract_values_to_points(fire_dataset_points, dataset_terrain, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\dem.csv


## World Settlement Footprint 2015 
- 2015-2016
- 10m 
- if want to use, may need to consider like (within 500m) got settlement boh? - future 

In [92]:
dataset_wsf = ee.Image("DLR/WSF/WSF2015/v1").select('settlement')
# dataset_wsf = dataset_wsf.clip(feature_selected_states)
dataset_wsf = dataset_wsf.unmask(0)

# opacity = 0.75
# blackBackground = ee.Image(0)
# Map.addLayer(blackBackground, None, "Black background", True, opacity)

# visualization = {
#   'min': 0,
#   'max': 255,
# }
# Map.addLayer(dataset_wsf, visualization, "Human settlement areas")

In [93]:
work_dir = os.path.expanduser('')
out_csv = os.path.join(work_dir, 'wsf.csv')
geemap.extract_values_to_points(fire_dataset_points, dataset_wsf, out_csv, geometries=True, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to D:\geemap_phd\wsf.csv


In [94]:
### Process dataset / rename column name
df = geemap.csv_to_df('wsf.csv')
df.set_index('system:index', inplace=True)

# rename the column first to esa_class_values
df = df.rename(columns={'first':'world_settlement_footprint'})
df.to_csv("wsf.csv")

# Combined CSV Module
- some codes to combine all the csv into a single csv file
- working already, but may need some cleanup of the codes 

In [ ]:
Map

In [95]:
### list of csv names 
### combined based on the column 
### Read each CSV file into separate pandas DataFrames
terraclimate_df = pd.read_csv('terraclimate.csv')
ndvi_evi_df = pd.read_csv('ndvi_evi.csv')
landsurfacetemperature_df = pd.read_csv('landsurfacetemperature.csv')
esa_df = pd.read_csv('esa.csv')
dem_df = pd.read_csv('dem.csv')
KBDI_df = pd.read_csv('KBDI.csv')
MCD12Q1_df = pd.read_csv('MCD12Q1.csv')
wsf_df = pd.read_csv('wsf.csv')
hii_df = pd.read_csv('hii.csv')
nighttime_df = pd.read_csv('nighttime.csv')
fire_dataset = pd.read_csv('fire_dataset.csv')


## join the csv files, then duplicate column name will have suffix of DROP, filter away any column that contain DROP 
merged_df = terraclimate_df.join(ndvi_evi_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(landsurfacetemperature_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(esa_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(dem_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(KBDI_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(MCD12Q1_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(wsf_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(hii_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(nighttime_df, lsuffix="DROP").filter(regex="^(?!.*DROP)")
merged_df = merged_df.join(fire_dataset, lsuffix="DROP").filter(regex="^(?!.*DROP)")

### Print the combined DataFrame (optional)
# print(merged_df)

### Save the combined DataFrame to a new CSV file
merged_df.set_index('system:index', inplace=True)
merged_df.to_csv('combined_data.csv', index=True)


## Filter for Journal - "current-annual"l
- simple filter only
- filter current-annual attributes for analysis
- the dataset after filtered will be analyzed with Fir
- For thesis purpose, refer to the other analysis notebook

In [99]:
### Filter any columns containing the "current" keyword
### Filter again with "annual keywords"
### Join with the fire_dataset (points) to get back the years / month of fire occurence 
merged_df = df.filter(regex=r'current')
merged_df = merged_df.filter(regex=r'annual')
merged_df = merged_df.join(fire_dataset, lsuffix="DROP").filter(regex="^(?!.*DROP)")

In [100]:
### Drop all columns containing only NaN values
merged_df = merged_df.dropna(axis=1, how='all')

In [101]:
merged_df.to_csv('combined_data_filtered_current_annual_nan.csv', index=True)

In [102]:
merged_df

,current_aet_annual,current_def_annual,current_pdsi_annual,current_pet_annual,current_pr_annual,current_ro_annual,current_soil_annual,current_srad_annual,current_swe_annual,current_tmmn_annual,...,ADM0_EN,ADM1_EN,ADM2_EN,validOn,Shape_Area,ADM0_PCODE,BurnDate,year,month,day
system:index,,,,,,,,,,,,,,,,,,,,,
1_2001_09_01_00000000000000000071_0,107.900000,1.591667,1.225000,109.483333,296.416667,188.666667,111.983333,188.691667,0.0,23.625000,...,Malaysia,Johor,Segamat,1613030400000,0.233776,MY,270,2001,9,27
1_2001_09_01_00000000000000000071_1,107.900000,1.591667,1.225000,109.483333,296.416667,188.666667,111.983333,188.691667,0.0,23.625000,...,Malaysia,Johor,Segamat,1613030400000,0.233776,MY,256,2001,9,13
1_2001_09_01_00000000000000000014_0,107.300000,1.483333,1.194167,108.791667,294.416667,187.166667,111.141667,187.225000,0.0,23.541667,...,Malaysia,Terengganu,Dungun,1613030400000,0.219613,MY,264,2001,9,21
1_2001_09_01_00000000000000000014_1,107.300000,1.483333,1.194167,108.791667,294.416667,187.166667,111.141667,187.225000,0.0,23.541667,...,Malaysia,Terengganu,Dungun,1613030400000,0.219613,MY,264,2001,9,21
1_2001_09_01_00000000000000000014_2,100.983333,0.916667,1.966667,101.900000,204.416667,103.333333,86.308333,173.558333,0.0,22.841667,...,Malaysia,Terengganu,Dungun,1613030400000,0.219613,MY,257,2001,9,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2_5645,95.875000,0.000000,4.268333,95.875000,337.166667,241.250000,107.600000,170.191667,0.0,21.450000,...,Malaysia,Terengganu,Hulu Terengganu,1613030400000,0.316419,MY,-1,2023,12,31
2_5646,97.558333,0.000000,1.896667,97.558333,250.000000,152.500000,96.700000,168.191667,0.0,21.666667,...,Malaysia,Negeri,Jelebu,1613030400000,0.110271,MY,-1,2023,12,31
2_5647,85.125000,0.000000,3.253333,85.125000,334.166667,249.166667,113.700000,168.666667,0.0,17.866667,...,Malaysia,Pahang,Raub,1613030400000,0.186307,MY,-1,2023,12,31
